In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
!pip install gdown

In [3]:
import time
import json
import zipfile
import subprocess as sp
from pathlib import Path 

import cv2
import gdown
import pandas as pd
import sqlalchemy as db 
import plotly.graph_objects as go 
from tqdm import tqdm 

### Download Sample Videos

In [4]:
video_dir = Path('sample_videos')
if not video_dir.exists():
    video_dir.mkdir()

In [5]:
files = [x for x in video_dir.iterdir()]
id_ = '1SCfmy1iql3Z3Nfi4EeReHJUYhB6KY3Pz'
url = f"https://drive.google.com/uc?id={id_}"
if not files:
    gdown.download(url)

In [6]:
zip_src = 'sample_videos.zip'
zip_dst = 'sample_videos'
with zipfile.ZipFile(zip_src, 'r') as f:
    for member in f.infolist():
    # Only extract files (not directories)
        if not member.is_dir():
            # Get just the filename (no path)
            filename = os.path.basename(member.filename)
            if filename:  # skip empty names (from directories)
                target = os.path.join(zip_dst, filename)
                # Write the file out
                with f.open(member) as source, open(target, 'wb') as dest:
                    dest.write(source.read())

In [7]:
scales = [
    '480',
    '720',
    '1080', 
    '2160'
]
codecs = [
    'libx264',
    'libx265'
]

In [8]:
for video in video_dir.iterdir():
    print(str(video))
    command = [
        'ffprobe',
        '-hide_banner',
        '-i', str(video),
        '2>&1', '|', 'grep', 'Video:'
        ]
    sp.run(f'ffprobe -hide_banner -i {str(video)} 2>&1 | grep Video:', shell=True)
    

sample_videos/libx265_720.mp4
  Stream #0:0[0x1](und): Video: hevc (Main) (hev1 / 0x31766568), yuv420p(tv, progressive), 1280x720 [SAR 1:1 DAR 16:9], 1703 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
sample_videos/libx264_720.mp4
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1280x720 [SAR 1:1 DAR 16:9], 2194 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
sample_videos/libx264_480.mp4
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 854x480 [SAR 1280:1281 DAR 16:9], 992 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
sample_videos/libx264_1080.mp4
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1920x1080 [SAR 1:1 DAR 16:9], 6146 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
sample_videos/libx265_480.mp4
  Stream #0:0[0x1](und): Video: hevc (Main) (hev1 / 0x31766568), yuv420p(tv, progressive), 854x480 [SAR 1280:1281 DAR 16:9], 865 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
sample_videos/lib

In [9]:
benchmarks = {}
videos = [x for x in video_dir.iterdir()]
videos

[PosixPath('sample_videos/libx265_720.mp4'),
 PosixPath('sample_videos/libx264_720.mp4'),
 PosixPath('sample_videos/libx264_480.mp4'),
 PosixPath('sample_videos/libx264_1080.mp4'),
 PosixPath('sample_videos/libx265_480.mp4'),
 PosixPath('sample_videos/libx265_1080.mp4'),
 PosixPath('sample_videos/libx264_2160.mp4'),
 PosixPath('sample_videos/libx265_2160.mp4')]

In [10]:
benchmarks['python_base'] = {}
for video in tqdm(videos):
    print('\n---------------------------------------------------')
    print(f'Finding faces in {str(video)}')
    print('-----------------------------------------------------\n\n')
    codec, res = video.stem.split('_')
    cap = cv2.VideoCapture(str(video))
    framecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    t = time.time()
    command = ["python",
               "find_faces_base.py",
               str(video),
               "--frameskip", "1"
    ]
    sp.run(command)
    duration = time.time() - t 
    fps = framecount / duration
    if codec not in benchmarks['python_base'].keys():
        benchmarks['python_base'][codec] = {}
    else:
        benchmarks['python_base'][codec][res] = round(fps, 1)
  

  0%|          | 0/8 [00:00<?, ?it/s]


--------------------------------------
Finding faces in sample_videos/libx265_720.mp4
--------------------------------------




E0000 00:00:1750000081.613956  325371 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750000081.658361  325371 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750000082.025151  325371 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750000082.025174  325371 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750000082.025176  325371 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750000082.025179  325371 computation_placer.cc:177] computation placer already registered. Please check linka

118.11028361320496


 12%|█▎        | 1/8 [02:09<15:03, 129.13s/it]

...

 88%|████████▊ | 7/8 [13:41<02:01, 121.71s/it]


--------------------------------------
Finding faces in sample_videos/libx265_2160.mp4
--------------------------------------




E0000 00:00:1750000899.422870  439816 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750000899.428333  439816 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750000899.441006  439816 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750000899.441025  439816 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750000899.441027  439816 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750000899.441028  439816 computation_placer.cc:177] computation placer already registered. Please check linka

140.88581728935242


100%|██████████| 8/8 [16:05<00:00, 120.74s/it]


In [11]:
benchmarks['python_base']

{'libx265': {'480': 12.8, '1080': 12.8, '2160': 10.0},
 'libx264': {'480': 14.2, '1080': 12.7, '2160': 10.2}}

In [12]:
name = 'python_threaded'
benchmarks[name] = {}
videos = [x for x in video_dir.iterdir()]
for video in videos:
    codec, res = video.stem.split('_')
    cap = cv2.VideoCapture(str(video))
    framecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    t = time.time()
    command = ["python",
               "find_faces_threaded.py",
               str(video),
               "test.csv",
               "--frameskip", "1",
               "--num_threads", "1"
    ]
    sp.run(command)
    duration = time.time() - t 
    fps = framecount / duration
    if codec not in benchmarks[name].keys():
        benchmarks[name][codec] = {}
    else:
        benchmarks[name][codec][res] = round(fps, 1)

E0000 00:00:1750001043.755615  456152 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750001043.761233  456152 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750001043.774286  456152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750001043.774304  456152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750001043.774306  456152 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750001043.774307  456152 computation_placer.cc:177] computation placer already registered. Please check linka

In [13]:
benchmarks

{'python_base': {'libx265': {'480': 12.8, '1080': 12.8, '2160': 10.0},
  'libx264': {'480': 14.2, '1080': 12.7, '2160': 10.2}},
 'python_threaded': {'libx265': {'480': 15.8, '1080': 15.0, '2160': 11.9},
  'libx264': {'480': 16.5, '1080': 15.3, '2160': 11.9}}}

In [14]:
name = 'visage'
benchmarks[name] = {}
videos = [x for x in video_dir.iterdir()]
for video in videos:
    print(f'Analyzing {str(video)}')
    codec, res = video.stem.split('_')
    cap = cv2.VideoCapture(str(video))
    framecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    t = time.time()
    command = ["../bin/main",
               str(video),
               "test.csv",
               "1"
    ]
    sp.run(command)
    duration = time.time() - t 
    fps = framecount / duration
    if codec not in benchmarks[name].keys():
        benchmarks[name][codec] = {}
    else:
        benchmarks[name][codec][res] = round(fps, 1)

Analyzing sample_videos/libx265_720.mp4
Analyzing sample_videos/libx264_720.mp4
Analyzing sample_videos/libx264_480.mp4
Analyzing sample_videos/libx264_1080.mp4
Analyzing sample_videos/libx265_480.mp4
Analyzing sample_videos/libx265_1080.mp4
Analyzing sample_videos/libx264_2160.mp4


../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory
../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory
../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory
../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory
../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory
../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory


Analyzing sample_videos/libx265_2160.mp4


../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory
../bin/main: error while loading shared libraries: libopencv_highgui.so.412: cannot open shared object file: No such file or directory


In [15]:
resolutions = sorted({r for impl in benchmarks.values() for codec in impl.values() for r in codec})
resolutions = [resolutions[2], resolutions[0], resolutions[1]]
resolutions

['480', '1080', '2160']

In [16]:
layout = go.Layout(
    title=dict(
        text='FPS by Resolution (libx264)', 
        font=dict(color='white'),
        x=0.5),
    plot_bgcolor="#4D555E",
    paper_bgcolor='rgba(61, 61, 61, 0)',
    yaxis=dict(title=dict(text='FPS', font=dict(color='white')), showgrid=False, tickfont=dict(color='white')),
    xaxis=dict(showgrid=True, tickfont=dict(color='white')),
    legend=dict(
        font=dict(color='white')
    ),
    margin=dict(
        l=20,
        r=20,
        t=40,
        b=20
    )
)

In [17]:
fig = go.Figure(layout=layout)
colors = ['#4ECDC4', '#416165', '#0B3948']
xaxis = ['Python', 'Python Threaded', 'Visage']
for num, res in enumerate(resolutions):
    y = []
    for k, v in benchmarks.items():
        y.append(benchmarks[k]['libx264'][res])
    fig.add_trace(go.Bar(
        y=xaxis, 
        x=y, name=res, 
        marker_color=colors[num],
        text=y,
        textposition='outside',
        orientation='h'))
fig.update_traces(textfont=dict(color='white'))
fig.show()

In [18]:
fig.write_image('libx264_benchmark.svg', width=800, height=400)

In [19]:
fig = go.Figure(layout=layout)
colors = ['#4ECDC4', '#416165', '#0B3948']
xaxis = ['Python', 'Python Threaded', 'Visage']
for num, res in enumerate(resolutions):
    y = []
    for k, v in benchmarks.items():
        y.append(benchmarks[k]['libx265'][res])
    fig.add_trace(go.Bar(
        y=xaxis, 
        x=y, name=res, 
        marker_color=colors[num],
        text=y,
        textposition='outside',
        orientation='h'))
fig.update_traces(textfont=dict(color='white'))
fig.update_layout(title=dict(
        text='FPS by Resolution (libx265)', 
        font=dict(color='white'),
        x=0.5))
fig.show()

In [20]:
fig.write_image('libx265_benchmark.svg', width=800, height=400)

In [21]:
methods = ['python_base', 'python_threaded', 'c++']
xaxis = ['Python', 'Python Threaded', 'Visage']
resolutions = ['480', '1080', '2160']
colors = ['#4ECDC4', '#416165', '#0B3948']

averaged = {}
for method in methods:
    averaged[method] = {}
    for res in resolutions:
        v1 = benchmarks[method]['libx265'][res]
        v2 = benchmarks[method]['libx264'][res]
        averaged[method][res] = (v1 + v2) / 2

KeyError: 'c++'

In [ ]:
fig = go.Figure(layout=layout)
for num, res in enumerate(resolutions):
    y = []
    for method in methods:
        y.append(averaged[method][res])
    fig.add_trace(go.Bar(
        y=xaxis, 
        x=y, name=res, 
        marker_color=colors[num],
        text=[f"{v:.1f}" for v in y],
        textposition='outside',
        orientation='h'
    ))
fig.update_traces(textfont=dict(color='white'))
fig.update_layout(
    title=dict(
        text='FPS by Resolution', 
        font=dict(color='white'),
        x=0.5
    ),
    barmode='group'
)
fig.show()

In [ ]:
fig.write_image('benchmarks.svg', width=800, height=400)